In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!ls '/content/gdrive/My Drive/Colab Notebooks/receipts-by-item-2017-01-05-2020-05-09.csv'

'/content/gdrive/My Drive/Colab Notebooks/receipts-by-item-2017-01-05-2020-05-09.csv'


In [ ]:
import pandas as pd
import csv

In [ ]:
path = '/content/gdrive/My Drive/Colab Notebooks/'
filename = 'receipts-by-item-2017-01-05-2020-05-09.csv'

df = pd.read_csv(path+filename)
df.columns = [c.replace(' ', '_') for c in df.columns]
df = df.drop(df[df.Item == '.'].index)
df = df.drop(df[df.Item == 'DL ค่าบริการจัดส่ง Zone 1'].index)
df = df.drop(df[df.Item == 'DL ค่าบริการจัดส่ง Zone 2'].index)
df = df.drop(df[df.Item == 'DL ค่าบริการจัดส่ง Zone 3'].index)
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Receipt_number,Receipt_type,Category,SKU,Item,Variant,Modifiers_applied,Quantity,Gross_sales,Discounts,Net_sales,Cost_of_goods,Gross_profit,Taxes,Dining_option,POS,Store,Cashier_name,Customer_name,Customer_contacts,Comment,Status
0,5/9/20 8:25 PM,12-10079,Sale,ซาชิมิ,10012.0,หอยนางรมพอนซึ,NaN,NaN,1.0,70.0,0.0,70.0,0.0,70.0,0.0,ทานที่ร้าน,iPad,ร้านอาหารญี่ปุ่นคิโมจิ l KIMOCHI IZAKAYA,staff,NaN,NaN,NaN,Closed
1,5/9/20 8:25 PM,12-10079,Sale,ข้าว/ชุดข้าว,10133.0,ข้าวหน้าเนื้อ,NaN,ไข่ออนเซน,1.0,150.0,0.0,150.0,0.0,150.0,0.0,ทานที่ร้าน,iPad,ร้านอาหารญี่ปุ่นคิโมจิ l KIMOCHI IZAKAYA,staff,NaN,NaN,NaN,Closed
2,5/9/20 8:25 PM,12-10079,Sale,เครื่องดื่ม,10054.0,DK น้ำเปล่า,NaN,NaN,1.0,15.0,0.0,15.0,6.0,9.0,0.0,ทานที่ร้าน,iPad,ร้านอาหารญี่ปุ่นคิโมจิ l KIMOCHI IZAKAYA,staff,NaN,NaN,NaN,Closed
3,5/9/20 8:25 PM,12-10079,Sale,ข้าว/ชุดข้าว,10169.0,ชุดข้าวหมูตุ๋นคัตสึโทจิ,NaN,NaN,1.0,145.0,0.0,145.0,0.0,145.0,0.0,ทานที่ร้าน,iPad,ร้านอาหารญี่ปุ่นคิโมจิ l KIMOCHI IZAKAYA,staff,NaN,NaN,NaN,Closed
4,5/9/20 8:21 PM,12-10078,Sale,ซาชิมิ,10007.0,แซลมอนซาชิมิ L,NaN,NaN,1.0,315.0,0.0,315.0,0.0,315.0,0.0,ทานที่ร้าน,iPad,ร้านอาหารญี่ปุ่นคิโมจิ l KIMOCHI IZAKAYA,staff,NaN,NaN,NaN,Closed


In [ ]:
df.shape

(88401, 23)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
#remove Receipt with 1 item
df = df[df.duplicated(subset=['Receipt_number'],keep=False)]

In [ ]:
df.shape

(87293, 23)

In [ ]:
basket = (df.groupby(['Receipt_number', 'Item'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Receipt_number'))
basket.head()

Item,ASH ค่าบริการ,CS หมอนซูชิ,DK KRN,DK ชามะลิ (รีฟิล),DK ชามะลิร้อน,DK ชาเขียว (รีฟิล),DK ชาเขียวร้อน,DK น้ำเปล่า,DK น้ำแข็ง,DK สาเก,DK โค้ก,DK โซดา,DK ไอศกรีมชาเขียวถั่วแดง,DS คาราเมลคัสตาร์ด,HNK ค่าบริการ,Heineken,LO ค่าบริการ,PM เยลลี่ JelloShot,SIN ค่าบริการ,Sushi Set,Sushi Set A,Sushi Set B,Sushi Set C,กาแฟ,กิมจิ,กุ้งทอดครีมวาซาบิ,กุ้งย่าง,กุ้งเบิร์นซอสไข่เค็ม,ข้าว,ข้าวปลาดิบรวมชิราชิ,ข้าวปลาดิบไข่กุ้ง,ข้าวปลาดิบไข่แซลมอน,ข้าวปลาสุกย่างซอส,ข้าวผัดกระเทียม,ข้าวราดแกงกะหรี่,ข้าวหน้าปลาไหล,ข้าวหน้าหมู,ข้าวหน้าหมูผัดกิมจิ,ข้าวหน้าหมูย่างเกลือ,ข้าวหน้าหมูเกาหลี,...,แซลมอนยำไทย,แซลมอนยูเกะ,แซลมอนย่าง,แซลมอนราดซอสไข่ปลา,แซลมอนราดพริก,แซลมอนสไปซ์เทมากิ,แซลมอนฮามาจิพอนซึ,แซลมอนเทริยากิ,แซลมอนเบิร์นชีส,แซลมอนเบิร์นซอสปลาไหล,แซลมอนเบิร์นซอสไข่ปลา,แซลมอนเบิร์นซอสไข่เค็ม,แซลมอนเบิร์นมิโซะ,แซลมอนเบิร์นเกลือเลมอน,แซลมอนเบิร์นเมนไทโกะ,แซลมอนโทโร่พอนซึ,แซลมอนโรล,แซลมอนไซส์ S,โทโร่ซาชิมิ,โนริทามาโกะ,โนริทูน่ามิโซะ,โนริปูอัด,โนริมันปูมิโซะ,โนริสาหร่าย,โนริหอยนางรม,โนริอิคุระ,โนริเมนไทโกะ,โนริเอบิโกะ,โนริแซลมอน,โนริแซลมอนเผ็ด,โอโคโนมิยากิ,ไก่คาราเกะ,ไก่ย่างเทริยากิ,ไข่กุ้ง,ไข่ดาวชีส,ไข่ตุ๋นญี่ปุ่น,ไข่หวาน,ไข่ออนเซน,ไอศกรีมมัทฉะ,ไฮ
Receipt_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1-1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-1003,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-1004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-1006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
#basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets.head()

Item,ASH ค่าบริการ,CS หมอนซูชิ,DK KRN,DK ชามะลิ (รีฟิล),DK ชามะลิร้อน,DK ชาเขียว (รีฟิล),DK ชาเขียวร้อน,DK น้ำเปล่า,DK น้ำแข็ง,DK สาเก,DK โค้ก,DK โซดา,DK ไอศกรีมชาเขียวถั่วแดง,DS คาราเมลคัสตาร์ด,HNK ค่าบริการ,Heineken,LO ค่าบริการ,PM เยลลี่ JelloShot,SIN ค่าบริการ,Sushi Set,Sushi Set A,Sushi Set B,Sushi Set C,กาแฟ,กิมจิ,กุ้งทอดครีมวาซาบิ,กุ้งย่าง,กุ้งเบิร์นซอสไข่เค็ม,ข้าว,ข้าวปลาดิบรวมชิราชิ,ข้าวปลาดิบไข่กุ้ง,ข้าวปลาดิบไข่แซลมอน,ข้าวปลาสุกย่างซอส,ข้าวผัดกระเทียม,ข้าวราดแกงกะหรี่,ข้าวหน้าปลาไหล,ข้าวหน้าหมู,ข้าวหน้าหมูผัดกิมจิ,ข้าวหน้าหมูย่างเกลือ,ข้าวหน้าหมูเกาหลี,...,แซลมอนยำไทย,แซลมอนยูเกะ,แซลมอนย่าง,แซลมอนราดซอสไข่ปลา,แซลมอนราดพริก,แซลมอนสไปซ์เทมากิ,แซลมอนฮามาจิพอนซึ,แซลมอนเทริยากิ,แซลมอนเบิร์นชีส,แซลมอนเบิร์นซอสปลาไหล,แซลมอนเบิร์นซอสไข่ปลา,แซลมอนเบิร์นซอสไข่เค็ม,แซลมอนเบิร์นมิโซะ,แซลมอนเบิร์นเกลือเลมอน,แซลมอนเบิร์นเมนไทโกะ,แซลมอนโทโร่พอนซึ,แซลมอนโรล,แซลมอนไซส์ S,โทโร่ซาชิมิ,โนริทามาโกะ,โนริทูน่ามิโซะ,โนริปูอัด,โนริมันปูมิโซะ,โนริสาหร่าย,โนริหอยนางรม,โนริอิคุระ,โนริเมนไทโกะ,โนริเอบิโกะ,โนริแซลมอน,โนริแซลมอนเผ็ด,โอโคโนมิยากิ,ไก่คาราเกะ,ไก่ย่างเทริยากิ,ไข่กุ้ง,ไข่ดาวชีส,ไข่ตุ๋นญี่ปุ่น,ไข่หวาน,ไข่ออนเซน,ไอศกรีมมัทฉะ,ไฮ
Receipt_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1-1002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1-1003,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1-1004,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1-1005,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1-1006,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)
frequent_itemsets.shape

(45, 2)

In [ ]:
frequent_itemsets.head(10)

,support,itemsets
0,0.191140,(DK ชามะลิ (รีฟิล))
1,0.223714,(DK ชาเขียว (รีฟิล))
2,0.071752,(DK ชาเขียวร้อน)
3,0.516287,(DK น้ำเปล่า)
4,0.176449,(DK โค้ก)
5,0.070119,(Sushi Set A)
6,0.080285,(Sushi Set B)
7,0.070936,(กิมจิ)
8,0.086444,(ข้าว)
9,0.168806,(ข้าวราดแกงกะหรี่)


In [ ]:
rules = association_rules(frequent_itemsets, metric="lift",min_threshold=1)
rules.sort_values('confidence', ascending = False, inplace = True)
len(rules)

20

In [ ]:
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
7,(ชุดข้าวสเต๊กซาบะ),(DK น้ำเปล่า),0.122134,0.516287,0.076723,0.628190,1.216745,0.013667,1.300967
3,(ข้าว),(DK น้ำเปล่า),0.086444,0.516287,0.054241,0.627468,1.215347,0.009611,1.298446
15,(ยากิโซบะ),(DK น้ำเปล่า),0.098316,0.516287,0.059880,0.609057,1.179686,0.009121,1.237297
12,(ทาโกยากิ),(DK น้ำเปล่า),0.119982,0.516287,0.071529,0.596166,1.154718,0.009584,1.197801
5,(ข้าวราดแกงกะหรี่),(DK น้ำเปล่า),0.168806,0.516287,0.099503,0.589451,1.141711,0.012350,1.178209
18,(แซลมอนยำไทย),(DK น้ำเปล่า),0.107665,0.516287,0.060919,0.565817,1.095934,0.005333,1.114075
11,(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),(DK น้ำเปล่า),0.155450,0.516287,0.085108,0.547494,1.060445,0.004851,1.068965
8,(ซุปมิโซะ),(DK น้ำเปล่า),0.132077,0.516287,0.071678,0.542697,1.051153,0.003488,1.057751
16,(แซลมอนซาชิมิ L),(DK น้ำเปล่า),0.148549,0.516287,0.078430,0.527972,1.022633,0.001736,1.024755
1,(DK ชามะลิ (รีฟิล)),(DK ชาเขียว (รีฟิล)),0.191140,0.223714,0.058099,0.303960,1.358694,0.015338,1.115288


Remove Category เครื่องดื่ม

In [ ]:
df = df.drop(df[df.Category == 'เครื่องดื่ม'].index)
df = df.drop(df[df.Category == 'ท้อปปิ้ง'].index)
df = df[df.duplicated(subset=['Receipt_number'],keep=False)]
basket = (df.groupby(['Receipt_number', 'Item'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Receipt_number'))
basket_sets = basket.applymap(encode_units)
frequent_itemsets = apriori(basket_sets, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values('confidence', ascending = False, inplace = True)
len(rules)

10

In [ ]:
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ทาโกยากิ),(ข้าวราดแกงกะหรี่),0.123688,0.168262,0.024355,0.196904,1.170221,0.003543,1.035664
6,(แซลมอนยำไทย),(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),0.110822,0.159991,0.021061,0.190048,1.187871,0.003331,1.037110
8,(ทาโกยากิ),(แซลมอนซาชิมิ L),0.123688,0.151873,0.020372,0.164706,1.084501,0.001587,1.015364
4,(แซลมอนซาชิมิ M),(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),0.152102,0.159991,0.024891,0.163646,1.022843,0.000556,1.004370
2,(ทาโกยากิ),(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),0.123688,0.159991,0.020219,0.163467,1.021731,0.000430,1.004156
5,(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),(แซลมอนซาชิมิ M),0.159991,0.152102,0.024891,0.155577,1.022843,0.000556,1.004115
1,(ข้าวราดแกงกะหรี่),(ทาโกยากิ),0.168262,0.123688,0.024355,0.144743,1.170221,0.003543,1.024618
9,(แซลมอนซาชิมิ L),(ทาโกยากิ),0.151873,0.123688,0.020372,0.134140,1.084501,0.001587,1.012071
7,(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),(แซลมอนยำไทย),0.159991,0.110822,0.021061,0.131642,1.187871,0.003331,1.023976
3,(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),(ทาโกยากิ),0.159991,0.123688,0.020219,0.126376,1.021731,0.000430,1.003077


Seperate data to afternoon and evening

In [ ]:
#afternoon
basket_afternoon = (df[df['Date'].dt.hour <= 15] 
          .groupby(['Receipt_number', 'Item'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Receipt_number'))
basket_sets = basket_afternoon.applymap(encode_units)
frequent_itemsets = apriori(basket_sets, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values('confidence', ascending = False, inplace = True)
len(rules)

14

In [ ]:
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(ข้าว),(ซุปมิโซะ),0.093612,0.120393,0.025307,0.270341,2.245487,0.014037,1.205504
11,(ยำสาหร่าย),(ซุปมิโซะ),0.084275,0.120393,0.020639,0.244898,2.034152,0.010493,1.164885
2,(ข้าว),(แซลมอนซาชิมิ L),0.093612,0.139803,0.020639,0.220472,1.577017,0.007552,1.103484
0,(ซุปมิโซะ),(ข้าว),0.120393,0.093612,0.025307,0.210204,2.245487,0.014037,1.147623
4,(ทาโกยากิ),(ข้าวราดแกงกะหรี่),0.130467,0.189681,0.027027,0.207156,1.092132,0.002280,1.022042
12,(ซุปมิโซะ),(แซลมอนซาชิมิ L),0.120393,0.139803,0.024816,0.206122,1.474373,0.007984,1.083538
6,(แซลมอนซาชิมิ M),(ข้าวราดแกงกะหรี่),0.151597,0.189681,0.028993,0.191248,1.008263,0.000238,1.001938
8,(ซุปมิโซะ),(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),0.120393,0.153317,0.022604,0.187755,1.224621,0.004146,1.042399
13,(แซลมอนซาชิมิ L),(ซุปมิโซะ),0.139803,0.120393,0.024816,0.177504,1.474373,0.007984,1.069437
10,(ซุปมิโซะ),(ยำสาหร่าย),0.120393,0.084275,0.020639,0.171429,2.034152,0.010493,1.105185


In [ ]:
#evening
basket_evening = (df[df['Date'].dt.hour > 15] 
          .groupby(['Receipt_number', 'Item'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Receipt_number'))
basket_sets = basket_evening.applymap(encode_units)
frequent_itemsets = apriori(basket_sets, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values('confidence', ascending = False, inplace = True)
len(rules)

26

In [ ]:
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(กิมจิ),(ซุปมิโซะ),0.074387,0.141743,0.026920,0.361891,2.553153,0.016376,1.345000
2,(ข้าว),(ซุปมิโซะ),0.086035,0.141743,0.026371,0.306513,2.162464,0.014176,1.237598
9,(ยำสาหร่าย),(ซุปมิโซะ),0.084936,0.141743,0.025821,0.304010,2.144805,0.013782,1.233147
15,(แซลมอนซาชิมิ XL),(ซุปมิโซะ),0.098780,0.141743,0.023184,0.234705,1.655854,0.009183,1.121473
23,(แซลมอนยำไทย),(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),0.108340,0.160971,0.021866,0.201826,1.253798,0.004426,1.051185
10,(ซุปมิโซะ),(แซลมอนซาชิมิ L),0.141743,0.156466,0.027689,0.195349,1.248504,0.005511,1.048322
4,(ทาโกยากิ),(ข้าวราดแกงกะหรี่),0.119218,0.157785,0.022855,0.191705,1.214978,0.004044,1.041965
0,(ซุปมิโซะ),(กิมจิ),0.141743,0.074387,0.026920,0.189922,2.553153,0.016376,1.142622
17,(แซลมอนยำไทย),(ซุปมิโซะ),0.108340,0.141743,0.020547,0.189655,1.338025,0.005191,1.059126
3,(ซุปมิโซะ),(ข้าว),0.141743,0.086035,0.026371,0.186047,2.162464,0.014176,1.122872


Change Item Sushi Set A,B,C to Sushi Set

In [ ]:
df.loc[df['Item'].str.contains("Sushi Set"),'Item'] = 'Sushi Set'

In [ ]:
basket = (df.groupby(['Receipt_number', 'Item'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Receipt_number'))
basket_sets = basket.applymap(encode_units)
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values('confidence', ascending = False, inplace = True)
len(rules)

130

In [ ]:
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
91,(มากุโร่นิกิริ),(แซลมอนนิกิริ),0.033775,0.081183,0.015624,0.462585,5.698088,0.012882,1.709698
10,(ซาชิมิรวม 5 อย่าง),(Sushi Set),0.031937,0.184422,0.011794,0.369305,2.002496,0.005905,1.293141
8,(ชุดข้าวสเต็กปลาซาบะย่างเกลือ),(Sushi Set),0.033698,0.184422,0.012101,0.359091,1.947114,0.005886,1.272533
5,(คานิซาชิมิ),(Sushi Set),0.029410,0.184422,0.010033,0.341146,1.849809,0.004609,1.237873
115,(เอบินิกิริ),(แซลมอนนิกิริ),0.032933,0.081183,0.010416,0.316279,3.895902,0.007742,1.343849
3,(ข้าวไข่ข้นคิโมจิ),(Sushi Set),0.048556,0.184422,0.015317,0.315457,1.710518,0.006363,1.191420
64,(เทมปุระโรล),(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),0.041510,0.159991,0.012177,0.293358,1.833592,0.005536,1.188734
12,(ทามาโกะซาชิมิ),(Sushi Set),0.037987,0.184422,0.010569,0.278226,1.508636,0.003563,1.129963
58,(อูนากินิกิริ),(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),0.051620,0.159991,0.013786,0.267062,1.669235,0.005527,1.146085
52,(ทามาโกะซาชิมิ),(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),0.037987,0.159991,0.010033,0.264113,1.650800,0.003955,1.141492


In [ ]:
rules['antecedents'] = rules['antecedents'].astype(str)
rules['consequents'] = rules['consequents'].astype(str)

In [ ]:
rules.to_csv('/content/gdrive/My Drive/result_apriori.csv')

In [ ]:
df.loc[df['Item'].str.contains("แซลมอนซาชิมิ"),'Item'] = 'แซลมอนซาชิมิ'

Change Item แซลมอนซาชิมิ XL,M,L,S to แซลมอนซาชิมิ

In [ ]:
basket = (df.groupby(['Receipt_number', 'Item'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Receipt_number'))
basket_sets = basket.applymap(encode_units)

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values('confidence', ascending = False, inplace = True)
len(rules)

44

In [ ]:
#rules.head(10)
searchfor = ['Sushi Set','ซูชิเซ็ตแซลมอนเบิร์นนิกิริ']
rules[rules['antecedents'].astype(str).str.contains('|'.join(searchfor))]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Sushi Set),(ข้าวราดแกงกะหรี่),0.183357,0.167641,0.034546,0.188406,1.123864,0.003807,1.025585
4,(Sushi Set),(ทาโกยากิ),0.183357,0.122694,0.030218,0.164803,1.343208,0.007721,1.050419
23,(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),(ซุปมิโซะ),0.158606,0.135145,0.022853,0.144088,1.066171,0.001418,1.010448
2,(Sushi Set),(ซุปมิโซะ),0.183357,0.135145,0.025966,0.141615,1.047871,0.001186,1.007537
30,(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),(แซลมอนยำไทย),0.158606,0.109863,0.020879,0.131642,1.198242,0.003454,1.025081
28,(ซูชิเซ็ตแซลมอนเบิร์นนิกิริ),(ทาโกยากิ),0.158606,0.122694,0.020044,0.126376,1.030013,0.000584,1.004215


In [ ]:
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
21,(ยำสาหร่าย),(แซลมอนซาชิมิ),0.085395,0.469786,0.053841,0.630493,1.342085,0.013724,1.434923
24,(หอยนางรมพอนซึ),(แซลมอนซาชิมิ),0.052845,0.469786,0.033086,0.626087,1.332706,0.008260,1.418013
39,(แซลมอนราดซอสไข่ปลา),(แซลมอนซาชิมิ),0.043961,0.469786,0.025733,0.585366,1.246026,0.005081,1.278751
23,(หมึกย่างซอส),(แซลมอนซาชิมิ),0.036532,0.469786,0.021215,0.580713,1.236121,0.004052,1.264560
14,(ทามาโกะซาชิมิ),(แซลมอนซาชิมิ),0.037987,0.469786,0.020985,0.552419,1.175895,0.003139,1.184622
41,(โนริอิคุระ),(แซลมอนซาชิมิ),0.075975,0.469786,0.041051,0.540323,1.150145,0.005359,1.153447
18,(ท้องแซลมอนย่างเกลือ),(แซลมอนซาชิมิ),0.043731,0.469786,0.023436,0.535902,1.140735,0.002891,1.142460
26,(อูนากินิกิริ),(แซลมอนซาชิมิ),0.051620,0.469786,0.027571,0.534125,1.136952,0.003321,1.138102
6,(ข้าวหน้าปลาไหล),(แซลมอนซาชิมิ),0.074290,0.469786,0.039442,0.530928,1.130147,0.004542,1.130346
16,(ทาโกะวาซาบิ),(แซลมอนซาชิมิ),0.043961,0.469786,0.022823,0.519164,1.105106,0.002171,1.102691
